# Imports

In [1]:
import os
import shutil

from PAL import *
engine = Redshift_Connection('ssopariwalla')

In [2]:
schema = account_schema_mapping['MTS']

# Pull OI Alerts

In [5]:
query = f"""
SELECT DISTINCT
    event_table.event_id,
    vehicle_meta_data.vehicle_id,
    event_type,
    event_table.unique_id,
    event_table.pressure_date,
    CASE
        WHEN event_status.status = 'CLOSED' THEN event_status.ts_created
        ELSE NULL
    END as ts_closed
FROM 
    {schema}.event_table INNER JOIN {schema}.event_status
        ON event_table.event_id = event_status.event_id
    INNER JOIN {schema}.meta_data
        ON meta_data.unique_id = event_table.unique_id
    INNER JOIN {schema}.vehicle_meta_data
        ON vehicle_meta_data.vehicle_id = meta_data.vehicle_id
WHERE
    event_type = 'OI'
    AND event_table.pressure_date >= '2025-04-30 18:45'
    AND event_status.event_status_id IN (SELECT MAX(event_status_id) FROM {schema}.event_status GROUP BY event_id)
    AND meta_data.id IN (SELECT MAX(id) FROM {schema}.meta_data WHERE active = 1 GROUP BY unique_id)
ORDER BY
    event_table.event_id ASC
"""
OI_alerts = pd.read_sql(query,engine)

In [6]:
OI_alerts

,event_id,vehicle_id,event_type,unique_id,pressure_date,ts_closed
0,3262687,4084,OI,4120_1F650C,2025-05-01 15:40:31,2025-05-01 16:33:19
1,3262725,4212,OI,4486_2E1043,2025-05-01 16:40:03,NaT
2,3262755,4084,OI,4120_1F650C,2025-05-01 18:12:38,2025-05-01 18:31:38
3,3262855,4084,OI,4120_1F650C,2025-05-01 21:20:28,2025-05-01 22:46:34
4,3263434,4084,OI,4120_1F650C,2025-05-02 21:48:12,2025-05-02 22:31:26
5,3263954,10576,OI,24373_1006FE41_3,2025-05-04 22:18:41,NaT


# Plot Each OI Alert

In [9]:
folder = 'OI Alert Plots'

if os.path.isdir(folder):
    shutil.rmtree(folder)

os.makedirs(folder)


In [10]:
for index, row in OI_alerts.iterrows():
    query = f"""
    SELECT
        pressure_date,
        sensor_data.unique_id,
        set_point,
        pressure,
        speed
    FROM
        {schema}.sensor_data INNER JOIN {schema}.meta_data
            ON sensor_data.meta_data_id = meta_data.id
    WHERE
        sensor_data.unique_id = '{row['unique_id']}'
        AND pressure_date >= '{row['pressure_date'] - pd.Timedelta(hours=48)}'
        AND pressure_date <= '{row['pressure_date'] + pd.Timedelta(hours=48)}'
    ORDER BY
        pressure_date ASC
    """
    sensor_data = pd.read_sql(query, engine)

    # Create the plot
    fig = go.Figure()

    # Add pressure line (solid blue)
    fig.add_trace(go.Scatter(
        x=sensor_data['pressure_date'],
        y=sensor_data['pressure'],
        mode='lines',
        name='Pressure',
        line=dict(color='blue', width=2)
    ))

    # Add set_point line (dashed black)
    fig.add_trace(go.Scatter(
        x=sensor_data['pressure_date'],
        y=sensor_data['set_point'],
        mode='lines',
        name='Set Point',
        line=dict(color='black', width=2, dash='dash')
    ))

    # Add vertical red line at the alert's pressure_date
    fig.add_trace(go.Scatter(
        x=[row['pressure_date'], row['pressure_date']],
        y=[0, 145],
        mode='lines',
        name='Alert Time',
        line=dict(color='red', width=2, dash='dot')
    ))

    if not pd.isna(row['ts_closed']):
        # Add vertical red line at the alert's pressure_date
        fig.add_trace(go.Scatter(
            x=[row['ts_closed'], row['ts_closed']],
            y=[0, 145],
            mode='lines',
            name='Alert Time',
            line=dict(color='red', width=2)
        ))

    # Update layout
    fig.update_layout(
        title=f"Alert {row['event_id']}",
        xaxis_title="Pressure Date",
        yaxis_title="Pressure (PSI)",
        template="simple_white",
        yaxis=dict(range=[0, 140]),
        legend=dict(x=0, y=1)
    )

    fig.write_html(f"OI Alert Plots/Alert {row['event_id']}.html")


shutil.make_archive('OI_alert_plots', 'zip', 'OI Alert Plots')


'/home/sagemaker-user/SaroshSopariwalla/04_28_2025 Validating OI Alerts/OI_alert_plots.zip'

# Investigate OI Alert

In [11]:
OI_alerts

,event_id,vehicle_id,event_type,unique_id,pressure_date,ts_closed
0,3262687,4084,OI,4120_1F650C,2025-05-01 15:40:31,2025-05-01 16:33:19
1,3262725,4212,OI,4486_2E1043,2025-05-01 16:40:03,NaT
2,3262755,4084,OI,4120_1F650C,2025-05-01 18:12:38,2025-05-01 18:31:38
3,3262855,4084,OI,4120_1F650C,2025-05-01 21:20:28,2025-05-01 22:46:34
4,3263434,4084,OI,4120_1F650C,2025-05-02 21:48:12,2025-05-02 22:31:26
5,3263954,10576,OI,24373_1006FE41_3,2025-05-04 22:18:41,NaT


In [12]:
row = OI_alerts.iloc[5]

In [13]:
query = f"""
SELECT
    pressure_date,
    sensor_data.unique_id,
    set_point,
    pressure,
    speed
FROM
    {schema}.sensor_data INNER JOIN {schema}.meta_data
        ON sensor_data.meta_data_id = meta_data.id
WHERE
    sensor_data.unique_id = '{row['unique_id']}'
    AND pressure_date >= '{row['pressure_date'] - pd.Timedelta(hours=48)}'
    AND pressure_date <= '{row['pressure_date'] + pd.Timedelta(hours=48)}'
ORDER BY
    pressure_date ASC
"""
sensor_data = pd.read_sql(query, engine)

In [14]:
sensor_data

,pressure_date,unique_id,set_point,pressure,speed
0,2025-05-04 22:18:41,24373_1006FE41_3,100,120,None
1,2025-05-04 22:18:41,24373_1006FE41_3,100,120,None
2,2025-05-04 22:18:41,24373_1006FE41_3,100,120,None
